# SonarAI Technical Notebook

# Notebook Overview

In this notebook, we aim to investigate potential correlations between environmental variables derived from NCAR datasets—such as **Sea Surface Temperature** and **Mean Evaporation Rate**(tbd) — and the **abundance of fish observed within the water column**, as recorded by sonar.

The **sonar dataset** was collected by **NOAA Fisheries** during a research cruise conducted aboard the vessel *H.B. Bigelow* between **October and November 2019**. To facilitate analysis, we integrate this dataset with two NCAR environmental datasets (*[tbd]* and *[tbd]*), aligning them both **spatially** and **temporally**.

This integration enables us to examine statistical relationships between physical oceanographic conditions and fish distribution patterns.

## Notebook Structure

1. **Imports**
2. **Data Loading**
2. **Data Preprocessing**
3. **Data Visualization**


---

## Prerequisites
This section was inspired by [this template](https://github.com/alan-turing-institute/the-turing-way/blob/master/book/templates/chapter-template/chapter-landing-page.md) of the wonderful [The Turing Way](https://the-turing-way.netlify.app) Jupyter Book.

Following your overview, tell your reader what concepts, packages, or other background information they'll **need** before learning your material. Tie this explicitly with links to other pages here in Foundations or to relevant external resources. Remove this body text, then populate the Markdown table, denoted in this cell with `|` vertical brackets, below, and fill out the information following. In this table, lay out prerequisite concepts by explicitly linking to other Foundations material or external resources, or describe generally helpful concepts.

Label the importance of each concept explicitly as **helpful/necessary**.

| Concepts | Importance | Notes |
| --- | --- | --- |
| [Intro to Cartopy](https://foundations.projectpythia.org/core/cartopy/cartopy) | Necessary | |
| [Understanding of NetCDF](https://foundations.projectpythia.org/core/data-formats/netcdf-cf) | Helpful | Familiarity with metadata structure |
| Project management | Helpful | |

- **Time to learn**: estimate in minutes. For a rough idea, use 5 mins per subsection, 10 if longer; add these up for a total. Safer to round up and overestimate.
- **System requirements**:
    - Populate with any system, version, or non-Python software requirements if necessary
    - Otherwise use the concepts table above and the Imports section below to describe required packages as necessary
    - If no extra requirements, remove the **System requirements** point altogether

---

## Imports
Begin your body of content with another `---` divider before continuing into this section, then remove this body text and populate the following code cell with all necessary Python imports **up-front**:

In [18]:
import xarray as xr
import s3fs
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import requests
import io
import os
from datetime import datetime, timedelta

## Initializing the datasets

All datasets are accessed using the OSDF infrastructure

In [19]:
bucket_name = 'noaa-wcsd-zarr-pds'
ship_name = "Henry_B._Bigelow"
cruise_name = "HB1906"
sensor_name = "EK60"

# Accessing the NOAA HB1906 dataset using OSDF
s3_file_system = s3fs.S3FileSystem(anon=True)
zarr_store = f'{cruise_name}.zarr'
s3_zarr_store_path = f"{bucket_name}/level_2/{ship_name}/{cruise_name}/{sensor_name}/{zarr_store}"
store = s3fs.S3Map(root=s3_zarr_store_path, s3=s3_file_system, check=False)
cruise = xr.open_zarr(store=store, consolidated=None)
start_time = "2019-10-16T15:00:00"
end_time = "2019-10-16T23:30:00"
timeslice = slice(start_time, end_time)
depths=slice(10, 250)
cruise = cruise.sel(time=timeslice, depth=depths, drop=False)
cruise = cruise.sel(frequency=38000, method='nearest').compute()
cruise = cruise.where(cruise.depth < cruise.bottom, drop=True)
hm_timestamps = cruise.time.values.tolist()

# location of one specific buoy located on Georges Bank
target_lon = 360 - 66.546
target_lat = 41.088
print(f"Target coordinates: Longitude: {target_lon}, Latitude: {target_lat}")
# Accessing stationary buoy data from a specific buoy located on Georges Bank, sampled daily
bouy_data_day_before = xr.open_dataset(
    'https://data.rda.ucar.edu/d277007/avhrr_v2.1/2019/oisst-avhrr-v02r01.20191015.nc#mode=bytes', engine='netcdf4')
buoy_data_actual_day = xr.open_dataset(
    'https://data.rda.ucar.edu/d277007/avhrr_v2.1/2019/oisst-avhrr-v02r01.20191016.nc#mode=bytes',
    engine='netcdf4')
buoy_data_day_after = xr.open_dataset(
    'https://data.rda.ucar.edu/d277007/avhrr_v2.1/2019/oisst-avhrr-v02r01.20191017.nc#mode=bytes',
    engine='netcdf4')

sst_day_before = bouy_data_day_before['sst'].sel(lon=target_lon, lat=target_lat, method='nearest').values[0][0]
sst_actual_day = buoy_data_actual_day['sst'].sel(lon=target_lon, lat=target_lat, method='nearest').values[0][0]
sst_day_after = buoy_data_day_after['sst'].sel(lon=target_lon, lat=target_lat, method='nearest').values[0][0]
# Downloading anomaly detection model from NCAR via OSDF
response = requests.get('https://data-osdf.rda.ucar.edu/ncar/rda/pythia_2025/osdf-cookbook/mae_error_map.npy')
response.raise_for_status()
error_map = np.load(io.BytesIO(response.content))

In [20]:
# This code has been copied from https://github.com/ioos/soundcoop/blob/main/3_HMD_environmental_data/plot_sound_environmental_and_climatology_data.ipynb in order to access buoy environmental data.
erddap_dataset = 'gov-ndbc-44005'
sound_dataset = 'Monh'
max_days = 25
start_date_time = '2019-10-16T14:00:00.000'
end_date_time = '2021-10-16T23:30:00.000'
min_frequency = 21
max_frequency = 24000

erddap_base_url = 'https://erddap.sensors.ioos.us/erddap'
# Update end_date_time if defined temporal range exceeds max_days
time_delta = datetime.fromisoformat(end_date_time) - datetime.fromisoformat(start_date_time)
if time_delta.days > max_days:
    end_date_time = str(datetime.fromisoformat(start_date_time) + timedelta(days=max_days))
    print(f'end_date_time updated to {end_date_time}')

# Get environmental sensor station lat/lon for use in mapping and querying water temperature climatology data
erddap_metadata_url = f'{erddap_base_url}/info/{erddap_dataset}/index.csv'
env_metadata_df = pd.read_csv(erddap_metadata_url)

env_station_x = env_metadata_df.loc[env_metadata_df['Attribute Name'] == 'geospatial_lon_min']['Value'].item()
env_station_y = env_metadata_df.loc[env_metadata_df['Attribute Name'] == 'geospatial_lat_min']['Value'].item()

wind_speed_units_row = env_metadata_df[
    (env_metadata_df['Row Type'] == 'attribute') &
    (env_metadata_df['Attribute Name'] == 'units') &
    (env_metadata_df['Variable Name'] == 'wind_speed')
    ]
wind_speed_units = wind_speed_units_row.iloc[0]['Value']
print(wind_speed_units)

wind_speed_to_kts_factors = {
    "m.s-1": 1.94384,
    "mph": 0.86897423357831,
    "kmh": 0.53995555554212126825,
    "ft.s-1": 0.59248243198521155506
}

if wind_speed_units in wind_speed_to_kts_factors:
    print("Success! Units can be converted from", wind_speed_units, 'to', 'kts')
else:
    print("Error! Wind speed cannot be converted from", wind_speed_units, 'to', 'kts')

wind_var = 'wind_speed'
swt_var = 'sea_surface_temperature'
wave_var = 'sea_surface_wave_significant_height'
anomaly_var = 'swt_anomaly'
wind_var_kts = 'wind_speed_kts'

erddap_dataset_url = (
    f'{erddap_base_url}/tabledap/{erddap_dataset}.csv'
    f'?time,{wind_var},{swt_var},{wave_var}'
)

env_df = pd.read_csv(
    erddap_dataset_url,
    skiprows=[1]  # The second row (index 1) are the column units, which we don't need
)

# Format the time field and set it as the index
env_df['time'] = pd.to_datetime(env_df['time'])
env_df['wind_speed_kts'] = env_df['wind_speed'].apply(lambda x: x * wind_speed_to_kts_factors[wind_speed_units])
del env_df['wind_speed']
env_df = env_df.set_index('time').sort_index()

#filtering by timestamps
env_df = env_df[(env_df.index > start_date_time) & (env_df.index < end_date_time)]
env_df.drop(env_df.tail(-9).index,inplace=True)
# env_df

---

## Preprocessing

In [21]:
def calculate_sv_mean(input_sv):
    sv = 10. ** (input_sv / 10.)
    return 10 * np.log10(np.mean(sv))

In [22]:
cruise['time_hour'] = cruise['time'].dt.floor('1h')

# Step 2: Group by each hour
grouped = cruise.groupby('time_hour')

# Step 3: Extract each 1-hour Dataset as a chunk
chunks = [group.drop_vars('time_hour') for _, group in grouped]

In [23]:
sv_hourly = []
timestamps = []

for i in range(0,len(chunks)):
    sv_data = chunks[i]['Sv']
    result = calculate_sv_mean(sv_data)

    ts = pd.to_datetime(chunks[i]['time'].values[0])
    result = result.compute()
    result = float(result.values)

    sv_hourly.append(result)

env_df["sv_hourly"] = sv_hourly

In [27]:
depths = np.asarray(cruise.depth.values)
times  = pd.to_datetime(hm_timestamps)

vals = error_map[:, :, 1]  # (1088, 28096)

n_depth = min(len(depths), vals.shape[0])
n_time  = min(len(times),  vals.shape[1])

df = pd.DataFrame(
    data=vals[:n_depth, :n_time],
    index=depths[:n_depth],
    columns=times[:n_time]
)

# Get current index from env_df
idx = env_df.index.tolist()

# Use the timestamps from df (columns) instead of hm_timestamps
df_timestamps = pd.to_datetime(df.columns).tz_localize(None)

# Replace first and last index entries in env_df
idx[0] = pd.Timestamp(df_timestamps.values[0], tz='UTC').floor("s")
idx[-1] = pd.Timestamp(df_timestamps.values[-1], tz='UTC').floor("s")

# Assign back to env_df
env_df.index = idx

## Visualization

Decription

In [28]:
def plot_synchronized_heatmaps_from_df(
    heatmap_df: pd.DataFrame,  # rows = depths, columns = timestamps
    line_df: pd.DataFrame,
    colorscale: str = "Reds",
    show_markers: bool = False,
    show: bool = True,
):
    # --- Validate line_df ---
    if not isinstance(line_df.index, pd.DatetimeIndex):
        raise TypeError("line_df must have a DatetimeIndex")
    line_df = line_df.copy()
    if line_df.index.tz is not None:
        line_df.index = line_df.index.tz_convert(None)

    # --- Extract axis values from heatmap_df ---
    depths = np.asarray(heatmap_df.index)
    heatmap_timestamps = pd.to_datetime(heatmap_df.columns)
    z = heatmap_df.to_numpy()

    # --- Build figure ---
    fig = make_subplots(
        rows=2,
        cols=1,
        shared_xaxes=True,
        vertical_spacing=0.03,
        row_heights=[0.3, 0.7]  # smaller line plot, bigger heatmap
    )

    # Top panel: line plots
    mode = "lines+markers" if show_markers else "lines"
    for col in line_df.columns:
        fig.add_trace(
            go.Scatter(
                x=line_df.index,
                y=line_df[col],
                name=str(col),
                mode=mode
            ),
            row=1, col=1
        )

    # Bottom panel: heatmap
    fig.add_trace(
        go.Heatmap(
            z=z,
            x=heatmap_timestamps,
            y=depths,
            colorscale=colorscale,
            zmin=np.nanmin(z),
            zmax=np.nanmax(z),
            colorbar_len=0.28,
            colorbar_y=0.14,
            hovertemplate="t=%{x}<br>depth=%{y}<br>value=%{z}<extra></extra>",
        ),
        row=2, col=1
    )

    fig.update_yaxes(autorange="reversed", row=2, col=1, title_text="Depth")
    fig.update_layout(
        legend_title_text="Signals",
        margin=dict(l=60, r=40, t=40, b=40),
        hovermode="x unified",
        template="plotly_white",
        height=700
    )

    # Save/show
    save_path = os.path.join(os.path.dirname(os.getcwd()), "out.html")
    fig.write_html(save_path)
    print(f"Plot saved to: {save_path}")
    if show:
        fig.show()

    return fig

# env_df = repeat_first_drop_last(env_df)
fig = plot_synchronized_heatmaps_from_df(heatmap_df=df, line_df=env_df)

# fig.show()